# Circuit learning module: Pennylane with gradient descent

In [15]:
import json
import os
import sys
import time
import glob
import warnings
import collections
from pathlib import Path
from pennylane import numpy as np
import pickle
import pennylane as qml
from sympy import default_sort_key
import torch
from discopy.quantum.pennylane import to_pennylane, PennyLaneCircuit
from inspect import signature
from noisyopt import minimizeSPSA
from utils import transform_into_pennylane_circuits, read_diagrams, get_symbols, create_labeled_classes, acc_from_dict, loss_from_dict

this_folder = os.path.abspath(os.getcwd())
nshot = 10000

torch.manual_seed(42)
np.random.seed(42)

# Plotting
import matplotlib.pyplot as plt

# OpenMP: number of parallel threads.
os.environ["OMP_NUM_THREADS"] = "1"

In [16]:
n_qubits = 4                # Number of qubits
step = 0.0004               # Learning rate
batch_size = 4              # Number of samples for each training step
num_epochs = 3              # Number of training epochs
q_depth = 6                 # Depth of the quantum circuit (number of variational layers)
gamma_lr_scheduler = 0.1    # Learning rate reduction applied every 10 epochs.
q_delta = 0.01              # Initial spread of random quantum weights
start_time = time.time()    # Start of the computation timer

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
# Select workload
#workload = "execution_time"
workload = "cardinality"

# Select workload size
#workload_size = "small"
#workload_size = "medium"
#workload_size = "large"
workload_size = "main"

classification = 2
layers = 1
single_qubit_params = 3
n_wire_count = 1

# Access the selected circuits
path_name = this_folder + "//simplified-JOB-diagrams//"\
            + workload + "//" + workload_size + "//circuits//"\
            + str(classification) + "//" + str(layers) + "_layer//"\
           + str(single_qubit_params) + "_single_qubit_params//" + str(n_wire_count)\
            + "_n_wire_count//"

In [3]:
training_circuits_paths = glob.glob(path_name + "training//[0-9]*.p")
validation_circuits_paths = glob.glob(path_name + "validation//[0-9]*.p")
test_circuits_paths = glob.glob(path_name + "test//[0-9]*.p")

In [4]:
training_circuits = read_diagrams(training_circuits_paths)
validation_circuits = read_diagrams(validation_circuits_paths)
test_circuits = read_diagrams(test_circuits_paths)

In [5]:
training_data, test_data, validation_data = None, None, None
data_path = this_folder + "//data//" + workload + "//" + workload_size + "//"

with open(data_path + "training_data.json", "r") as inputfile:
    training_data = json.load(inputfile)['training_data']
with open(data_path + "test_data.json", "r") as inputfile:
    test_data = json.load(inputfile)['test_data']
with open(data_path + "validation_data.json", "r") as inputfile:
    validation_data = json.load(inputfile)['validation_data']

training_data_labels = create_labeled_classes(training_data, classification, workload)
test_data_labels = create_labeled_classes(test_data, classification, workload)
validation_data_labels = create_labeled_classes(validation_data, classification, workload)

In [6]:
fix = []
for i in training_circuits:
    if i not in training_data_labels:
        fix.append(i)
        
for i in fix:
    training_circuits.pop(i, None)

fix = []
for i in validation_circuits:
    if i not in validation_data_labels:    
        fix.append(i)

for i in fix:
    validation_circuits.pop(i, None)

fix = []
for i in test_circuits:
    if i not in test_data_labels:
        fix.append(i)
        
for i in fix:
    test_circuits.pop(i, None)

In [7]:
print("Number of training circuits: ", len(training_circuits))
print("Number of validation circuits: ", len(validation_circuits))
print("Number of test circuits: ", len(test_circuits))

Number of training circuits:  440
Number of validation circuits:  108
Number of test circuits:  109


In [8]:
all_symbols = set([elem for c in training_circuits.values() for elem in c.free_symbols])
all_symbols = list(sorted(all_symbols, key=default_sort_key))

In [10]:
def pick_params(all_params, all_symbols, current_symbols):
    res = []
    for param, symbol in zip(all_params, all_symbols):
        if symbol in current_symbols:
            res.append(param)
    return res

In [11]:
def training_cost(all_params, circuits, labels):
    predictions = {}
    for c in circuits:
        disco_circuit = circuits[c]
        qml_circuit = to_pennylane(disco_circuit, probabilities = True)
        current_symbols = disco_circuit.free_symbols
        current_symbols = list(sorted(current_symbols, key=default_sort_key))
        params = pick_params(all_params, all_symbols, current_symbols)
        result = qml_circuit.eval(symbols=current_symbols, weights=params)
        print(result)
        predictions[c] = result
    loss = loss_from_dict(predictions, labels)
    return loss

In [12]:
#rng = np.random.default_rng(0)
#init_params_spsa = torch.Tensor([[t] for t in np.array(rng.random(len(all_symbols)))], requires_grad=True)

TypeError: new() received an invalid combination of arguments - got (list, requires_grad=bool), but expected one of:
 * (*, torch.device device)
      didn't match because some of the keywords were incorrect: requires_grad
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, *, torch.device device)
 * (object data, *, torch.device device)


In [ ]:
optimizer_hybrid = optim.Adam(model_hybrid.fc.parameters(), lr=step)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_hybrid, step_size=10, gamma=gamma_lr_scheduler)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 10000.0  # Large arbitrary number
    best_acc_train = 0.0
    best_loss_train = 10000.0  # Large arbitrary number
    print("Training started:")

    for epoch in range(num_epochs):

        # Each epoch has a training and validation phase
        for phase in ["train", "validation"]:
            #if phase == "train":
                # Set model to training mode
            #    model.train()
            #else:
                # Set model to evaluate mode
            #    model.eval()
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            #n_batches = dataset_sizes[phase] // batch_size
            #it = 0
            for inputs, labels in dataloaders[phase]:
                since_batch = time.time()
                batch_size_ = len(inputs)
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                # Track/compute gradient and make an optimization step only when training
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                # Print iteration results
                running_loss += loss.item() * batch_size_
                batch_corrects = torch.sum(preds == labels.data).item()
                running_corrects += batch_corrects
                print(
                    "Phase: {} Epoch: {}/{} Iter: {}/{} Batch time: {:.4f}".format(
                        phase,
                        epoch + 1,
                        num_epochs,
                        it + 1,
                        n_batches + 1,
                        time.time() - since_batch,
                    ),
                    end="\r",
                    flush=True,
                )
                it += 1

            # Print epoch results
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]
            print(
                "Phase: {} Epoch: {}/{} Loss: {:.4f} Acc: {:.4f}        ".format(
                    "train" if phase == "train" else "validation  ",
                    epoch + 1,
                    num_epochs,
                    epoch_loss,
                    epoch_acc,
                )
            )

            # Check if this is the best model wrt previous epochs
            if phase == "validation" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == "validation" and epoch_loss < best_loss:
                best_loss = epoch_loss
            if phase == "train" and epoch_acc > best_acc_train:
                best_acc_train = epoch_acc
            if phase == "train" and epoch_loss < best_loss_train:
                best_loss_train = epoch_loss

            # Update learning rate
            if phase == "train":
                scheduler.step()

    # Print final results
    model.load_state_dict(best_model_wts)
    time_elapsed = time.time() - since
    print(
        "Training completed in {:.0f}m {:.0f}s".format(time_elapsed // 60, time_elapsed % 60)
    )
    print("Best test loss: {:.4f} | Best test accuracy: {:.4f}".format(best_loss, best_acc))
    return model